# The Cleaning of the Toronto Postcode Data

Importing necessary libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd

Requesting the url page and parsing our the table from the html file

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML since that is where the table can be found
tr_elements = doc.xpath('//tr')

Parsing out the column names from the html

In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) in the col list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Adding each row from the html to the col list

In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Creating and cleaning the data frame

In [5]:
# Creating a dict from our col list
Dict={title:column for (title,column) in col}

# Creating a pandas dataframe from the dict
df=pd.DataFrame(Dict)

# Parsing out the data resulted in \n at the end of each entry in the Neighbourhoods columsn, so this removes that
df=df.rename(columns={"Neighbourhood\n": "Neighbourhood"})
df["Neighbourhood"]=df.Neighbourhood.str.replace("\n","")

# ignoring the rows with no borough assigned
df=df[df.Borough != 'Not assigned']

# Replacing the "Not assigned" in the Queen's park Borough to also be Queen's park
df["Neighbourhood"]=df.Neighbourhood.str.replace("Not assigned","Queen's Park")

# Resetting the index
toronto_zip_data=df.reset_index(drop=True)

# Merging neighbourhoods with the same postcode and joinging with a comma
toronto_zip_data=toronto_zip_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()

The first five rows of the dataset. 
I have sorted mine by postcode.

In [6]:
toronto_zip_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The shape of my dataset.

In [7]:
toronto_zip_data.shape

(103, 3)

Saving the dataset as a csv for later use.

In [8]:
toronto_zip_data.to_csv("toronto_zip_data.csv")